In [1]:
import cv2
import numpy as np

In [2]:
data = np.load('stereo.npy',allow_pickle=True).item()
Kl, Dl, Kr, Dr, R, T, img_size = data['Kl'], data['Dl'], data['Kr'], data['Dr'], \
                                 data['R'], data['T'], data['img_size']

In [3]:
cap_left = cv2.VideoCapture(1)
cap_left.set(cv2.CAP_PROP_AUTOFOCUS, 1)
cap_left.set(cv2.CAP_PROP_AUTO_WB, 3)
cap_right = cv2.VideoCapture(2)
cap_right.set(cv2.CAP_PROP_AUTOFOCUS, 1)
cap_right.set(cv2.CAP_PROP_AUTO_WB, 3)

True

In [4]:
R1, R2, P1, P2, Q, validRoi1, validRoi2 = cv2.stereoRectify(Kl, Dl, Kr, Dr, img_size, R, T)

In [5]:
xmap_left, ymap_left = cv2.initUndistortRectifyMap(Kl, Dl, R1, P1, img_size, cv2.CV_32FC1)
xmap_right, ymap_right = cv2.initUndistortRectifyMap(Kr, Dr, R2, P2, img_size, cv2.CV_32FC1)

In [6]:
while True:
    ret_left, frame_left = cap_left.read()
    ret_right, frame_right = cap_right.read()
    if not ret_left:
        break
    if not ret_right:
        break

    left_img_rectified = cv2.remap(frame_left, xmap_left, ymap_left, cv2.INTER_LINEAR)
    right_img_rectified = cv2.remap(frame_right, xmap_right, ymap_right, cv2.INTER_LINEAR)
    
    stereo_bm = cv2.StereoBM_create(32)
    dispmap_bm = stereo_bm.compute(cv2.cvtColor(left_img_rectified, cv2.COLOR_BGR2GRAY), 
                               cv2.cvtColor(right_img_rectified, cv2.COLOR_BGR2GRAY))
    disp_bm = cv2.normalize(dispmap_bm,None, alpha=0, beta=255,norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)

    stereo_sgbm = cv2.StereoSGBM_create(0, 32)
    dispmap_sgbm = stereo_sgbm.compute(left_img_rectified, right_img_rectified)
    disp_sbm = cv2.normalize(dispmap_sgbm,None, alpha=0, beta=255,norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    
    original = np.concatenate((frame_left, frame_right), axis=1)
    rectified = np.concatenate((left_img_rectified,right_img_rectified),axis=1)
    
    cv2.imshow('Actual', original)
    cv2.imshow('Recitified',rectified)
    cv2.imshow('StereoBM', disp_bm)
    cv2.imshow('StereoSBM',disp_sbm)
    
    k = cv2.waitKey(10)
    if k == 27:
        break

cap_left.release()
cap_right.release()
cv2.destroyAllWindows()